# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---

# <h1><center>Clase 12: Transformación de Datos</center></h1>

Este notebook continua el trabajo anterior, con el objetivo que los estudiantes del curso aprendan diferentes comandos y técnicas en Python y, en particular, en Pandas y con strings, para la transformación de datos.

## 1. Imputación de datos faltantes

Usaremos un set artificial (sintético) de datos para trabajar en imputación de datos faltantes.


In [ ]:
# lectura de datos
import numpy as np
import pandas as pd

df = pd.read_csv('data\ejemplo_imp.csv')
df

Identificación de datos faltantes

In [ ]:
df.isna()

Las funciones de `pandas` no permiten identificar en forma automática todo tipo de datos faltantes o erróneos.

In [ ]:
df = df.replace('*', np.nan)
df = df.replace('None', np.nan)
df = df.replace(' ', np.nan)
df = df.replace(-999, np.nan)

In [ ]:
df

In [ ]:
df.isna()

Ahora que hemos identificado correctamente los datos faltantes, podemos definir qué hacer sobre los datos que faltan.

In [ ]:
df.columns

In [ ]:
df.dropna(subset=['col2'])

Importante: recuerden que este método no elimina los datos en el DataFrame original y debemos asignarlo para que ello ocurra.

In [ ]:
df

Hagamos ahora una imputación de valores faltantes.

In [ ]:
df2 = df.copy()
df2

In [ ]:
df2['col2'] = df2['col2'].fillna(value='d')
df2

## 2 Limpieza de textos (strings)

La limpieza no sólo puede ocurrir a nivel de columnas o filas, también puede que necesitemos limpiar texto para poder procesarlo posteriormente.

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

url = "https://en.wikipedia.org/wiki/2023_Rugby_World_Cup_squads"
headers = {"User-Agent": "imt2200-class-notebook"}
page = requests.get(url, headers=headers, timeout=10).text
soup = bs(page)
tables = soup.find_all('table')
# procesar la tabla 20 que tiene la escuadra de Chile
table = tables[19]

In [ ]:
#creamos un DataFrame vacío con los títulos de la tabla
df = pd.DataFrame(columns = ['jugador','posicion','nacimiento','caps', 'club'])

# iterar sobre cada fila ('tr') para completar la información
for row in table.find_all('tr')[1::]:
    cols = row.find_all("td")
    #print(cols)
    cols = [col.text.strip() for col in cols]
    #print(cols)
    jugador = cols[0]
    posicion = cols[1]
    nacimiento = cols[2]
    caps = cols[3]
    club = cols[4]
    new_row = pd.DataFrame({'jugador': jugador, 'posicion': posicion, 'nacimiento': nacimiento,'caps':caps, 'club':club}, index=['jugador'])
    df = pd.concat([df, new_row], ignore_index=True)
df

In [ ]:
df.info()

In [ ]:
df['caps'] = df['caps'].astype('int')

In [ ]:
df.info()

In [ ]:
mean_caps = df['caps'].mean()
mean_caps

In [ ]:
df['caps'].describe()

In [ ]:
df['edad'] = df['nacimiento'].str.slice(-3,-1)
df['edad'] = df['edad'].astype('int')

In [ ]:
df.head()

In [ ]:
df['nacimiento'] = df['nacimiento'].str.slice(1,11)
df.head()

In [ ]:
df.info()

## 3 Variables categóricas

La posición de los jugadores corresponde a una variable categórica pues sólo puede tomar un conjunto de valores predeterminados. Supongamos que para nuestro análisis queremos dividir a los jugadores sólo entre `forwards` y `backs`.

In [ ]:
# Ver valores unicos presentes en el dataset
categorias = df['posicion'].unique()
categorias

In [ ]:
# Definir lista de categorías que queremos
cat_simple = ['forwards','backs']

# Cuántos registros están en la lista de categorías deseadas?
df['posicion'].isin(cat_simple).sum()

In [ ]:
# Mapeo de categorías originales a nuevas categorías
map_cats={'Hooker':'forwards',
          'Prop':'forwards',
          'Lock':'forwards',
          'Back row':'forwards',
          'Scrum-half':'backs',
          'Fly-half':'backs',
          'Centre':'backs',
          'Wing':'backs',
          'Fullback':'backs'}
df['posicion_simple'] = df['posicion'].map(map_cats)

In [ ]:
df